<a href="https://colab.research.google.com/github/marisout0/Maria-Mariana-Varela-Cavalcanti-Souto-ITP-2025-2/blob/main/palavras_cruzadas_Projeto_U2_ITP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get update
!apt-get install -y gcc

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:3 https://cli.github.com/packages stable InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [83.2 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,411 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:14 http://security

In [2]:
%%writefile palavras_cruzadas.c

#include <stdio.h>
#include <stdlib.h> // Para malloc(), free(), system()
#include <string.h> // Para strcmp(), strcpy(), strlen()
#include <ctype.h>  // Para toupper()

// MATRIZES E CONSTANTES
#define LINHAS 10
#define COLUNAS 10

// Estrutura para armazenar CADA palavra do jogo
//PONTEIROS
typedef struct {
    int x;           // Coluna inicial
    int y;           // Linha inicial
    char direcao;    // 'H' (Horizontal) or 'V' (Vertical)
    int numero;      // O número da dica (ex: 1, 2, 3...)
    int respondida;  // 0 = não, 1 = sim

    //  STRINGS e PONTEIROS
    char *palavra; // A resposta
    char *dica;    // A dica
} PalavraCruzada;


/*
 * Função para inicializar um tabuleiro (Matriz)
 * Preenche com o caractere '#'
 *REPETIÇÕES ANINHADAS e MATRIZES
 */
void inicializarTabuleiro(char tabuleiro[LINHAS][COLUNAS]) {
    int i, j;
    for (i = 0; i < LINHAS; i++) {       // Loop aninhado
        for (j = 0; j < COLUNAS; j++) {  // Loop aninhado
            tabuleiro[i][j] = '#';
        }
    }
}

/*
 * Função para exibir um tabuleiro (Matriz) na tela
 * REPETIÇÕES ANINHADAS e MATRIZES ---
 */
void exibirTabuleiro(char tabuleiro[LINHAS][COLUNAS]) {
    int i, j;
    printf("    "); // Espaçamento para os índices das colunas
    for (j = 0; j < COLUNAS; j++) {
        printf("%d ", j);
    }
    printf("\n    ");
    for (j = 0; j < COLUNAS; j++) {
        printf("--");
    }
    printf("\n");

    for (i = 0; i < LINHAS; i++) {       // Loop aninhado
        printf("%d | ", i); // Índice da linha
        for (j = 0; j < COLUNAS; j++) {  // Loop aninhado
            printf("%c ", tabuleiro[i][j]);
        }
        printf("\n");
    }
}

/*
 * Função auxiliar para criar e alocar dinamicamente uma palavra
 *  ALOCAÇÃO DINÂMICA (malloc) e STRINGS (strcpy)
 */
PalavraCruzada* criarPalavra(int numero, int x, int y, char dir, const char* palavra, const char* dica) {

    // 1. Aloca memória para a estrutura principal
    PalavraCruzada* p = (PalavraCruzada*) malloc(sizeof(PalavraCruzada));

    // 2. Aloca memória para a STRING da palavra (+1 para o '\0')
    p->palavra = (char*) malloc(strlen(palavra) + 1);
    strcpy(p->palavra, palavra); // Copia a string

    // 3. Aloca memória para a STRING da dica (+1 para o '\0')
    p->dica = (char*) malloc(strlen(dica) + 1);
    strcpy(p->dica, dica); // Copia a string

    p->numero = numero;
    p->x = x;
    p->y = y;
    p->direcao = dir;
    p->respondida = 0;

    return p; // Retorna o ponteiro para a estrutura alocada
}


/*
 * Coloca uma palavra (String) dentro de uma Matriz
 * --- 1. DEMONSTRAÇÃO DE MATRIZES, PONTEIROS e STRINGS ---
 */
void colocarPalavraNoTabuleiro(char tabuleiro[LINHAS][COLUNAS], PalavraCruzada* p) {
    int i;
    int len = strlen(p->palavra);

    if (p->direcao == 'H') {
        for (i = 0; i < len; i++) {
            tabuleiro[p->y][p->x + i] = p->palavra[i];
        }
    } else { // 'V'
        for (i = 0; i < len; i++) {
            tabuleiro[p->y + i][p->x] = p->palavra[i];
        }
    }
}

/*
 * Exibe a lista de dicas
 * --- 3. DEMONSTRAÇÃO DE STRINGS ---
 */
void exibirDicas(PalavraCruzada** bancoDePalavras, int qtdPalavras) {
    int i;
    printf("\n--- DICAS ---\n");
    for (i = 0; i < qtdPalavras; i++) {
        PalavraCruzada* p = bancoDePalavras[i];

        // Só exibe a dica se ainda não foi respondida
        if (!p->respondida) {
            printf("%d. (%c) [%d,%d]: %s\n",
                   p->numero, p->direcao, p->y, p->x, p->dica);
        }
    }
}

/*
 * Verifica se o jogador venceu
 * --- 2. DEMONSTRAÇÕES DE REPETIÇÕES ANINHADAS e MATRIZES ---
 */
int verificarVitoria(char tabuleiroUsuario[LINHAS][COLUNAS], char gabarito[LINHAS][COLUNAS]) {
    int i, j;
    for (i = 0; i < LINHAS; i++) {
        for (j = 0; j < COLUNAS; j++) {
            if (tabuleiroUsuario[i][j] != gabarito[i][j]) {
                return 0; // Ainda não venceu
            }
        }
    }
    return 1; // Venceu!
}

/*
 * Libera toda a memória alocada dinamicamente
 * --- 4. DEMONSTRAÇÃO DE ALOCAÇÃO DINÂMICA (free) e PONTEIROS ---
 */
void liberarMemoria(PalavraCruzada** bancoDePalavras, int qtdPalavras) {
    int i;
    for (i = 0; i < qtdPalavras; i++) {
        // Libera as strings alocadas
        free(bancoDePalavras[i]->palavra);
        free(bancoDePalavras[i]->dica);

        // Libera a estrutura
        free(bancoDePalavras[i]);
    }
    // Libera o "vetor de ponteiros"
    free(bancoDePalavras);
    printf("Memoria liberada com sucesso!\n");
}

// Função principal (Main)
int main() {
    // --- 1. DECLARAÇÃO DAS MATRIZES ---
    char gabarito[LINHAS][COLUNAS];
    char tabuleiroUsuario[LINHAS][COLUNAS];

    int qtdPalavras = 4;
    int palavrasRespondidas = 0;

    inicializarTabuleiro(gabarito);
    inicializarTabuleiro(tabuleiroUsuario);

    // ALOCAÇÃO DINÂMICA (Array de ponteiros para struct)
    PalavraCruzada** bancoDePalavras;
    bancoDePalavras = (PalavraCruzada**) malloc(qtdPalavras * sizeof(PalavraCruzada*));

    // ALOCAÇÃO DINÂMICA (chamando a função que usa malloc)
    // Criando as palavras do nosso jogo
    bancoDePalavras[0] = criarPalavra(1, 1, 1, 'H', "CASA", "Lugar onde se mora");
    bancoDePalavras[1] = criarPalavra(2, 1, 1, 'V', "COMPUTADOR", "Ferramenta de trabalho do DEV");
    bancoDePalavras[2] = criarPalavra(3, 3, 1, 'V', "STRING", "Tipo de dado para textos");
    bancoDePalavras[3] = criarPalavra(4, 1, 6, 'H', "MALLOC", "Funcao para alocar memoria");


    // Preenche o gabarito com as respostas corretas
    int i;
    for (i = 0; i < qtdPalavras; i++) {
        colocarPalavraNoTabuleiro(gabarito, bancoDePalavras[i]);
    }

    // Loop principal do Jogo
    int jogoRodando = 1;
    while (jogoRodando) {
        system("clear || cls"); // Limpa a tela (Linux || Windows)
        printf("=== JOGO DE PALAVRAS CRUZADAS (ITP - U2) ===\n\n");
        exibirTabuleiro(tabuleiroUsuario);
        exibirDicas(bancoDePalavras, qtdPalavras);

        // Checa vitória
        if (palavrasRespondidas == qtdPalavras) {
            printf("\nPARABENS! Voce completou o jogo!\n");
            jogoRodando = 0;
            continue; // Pula para o fim do loop
        }

        int escolhaNum;
        printf("\nDigite o numero da dica (ou 0 para sair): ");
        scanf("%d", &escolhaNum);

        if (escolhaNum == 0) {
            jogoRodando = 0;
            continue;
        }

        // Procura a palavra escolhida
        PalavraCruzada* palavraEscolhida = NULL;
        for (i = 0; i < qtdPalavras; i++) {
            if (bancoDePalavras[i]->numero == escolhaNum) {
                palavraEscolhida = bancoDePalavras[i];
                break;
            }
        }

        // Validação da entrada
        if (palavraEscolhida == NULL) {
            printf("Numero invalido. Tente novamente.\n");
        } else if (palavraEscolhida->respondida == 1) {
            printf("Voce ja acertou essa palavra.\n");
        } else {
            // --- 3. MANIPULAÇÃO DE STRINGS ---
            char palpite[50];
            printf("Digite sua resposta para a dica %d: ", escolhaNum);
            scanf("%s", palpite);

            // Converte palpite para maiúsculas (para comparar)
            for (i = 0; palpite[i]; i++) {
                palpite[i] = toupper(palpite[i]);
            }

            // Compara a string do usuário com a string do gabarito
            if (strcmp(palpite, palavraEscolhida->palavra) == 0) {
                printf("Resposta correta!\n");
                palavraEscolhida->respondida = 1;
                palavrasRespondidas++;
                // Coloca a palavra no tabuleiro do usuário
                colocarPalavraNoTabuleiro(tabuleiroUsuario, palavraEscolhida);
            } else {
                printf("Resposta errada. Tente novamente.\n");
            }
        }

        printf("Pressione Enter para continuar...");
        // Limpa o buffer de entrada
        while (getchar() != '\n');
        getchar(); // Espera o Enter
    }

    // --- 4. LIBERAÇÃO DE MEMÓRIA (free) ---
    // Garante que não há memory leaks
    liberarMemoria(bancoDePalavras, qtdPalavras);

    return 0;
}

Writing palavras_cruzadas.c


In [4]:
!gcc palavras_cruzadas.c -o palavras_cruzadas
!./palavras_cruzadas

=== JOGO DE PALAVRAS CRUZADAS (ITP - U2) ===

    0 1 2 3 4 5 6 7 8 9 
    --------------------
0 | # # # # # # # # # # 
1 | # # # # # # # # # # 
2 | # # # # # # # # # # 
3 | # # # # # # # # # # 
4 | # # # # # # # # # # 
5 | # # # # # # # # # # 
6 | # # # # # # # # # # 
7 | # # # # # # # # # # 
8 | # # # # # # # # # # 
9 | # # # # # # # # # # 

--- DICAS ---
1. (H) [1,1]: Lugar onde se mora
2. (V) [1,1]: Ferramenta de trabalho do DEV
3. (V) [1,3]: Tipo de dado para textos
4. (H) [6,1]: Funcao para alocar memoria

Digite o numero da dica (ou 0 para sair): 1
Digite sua resposta para a dica 1: casa
Resposta correta!
Pressione Enter para continuar...2
=== JOGO DE PALAVRAS CRUZADAS (ITP - U2) ===

    0 1 2 3 4 5 6 7 8 9 
    --------------------
0 | # # # # # # # # # # 
1 | # C A S A # # # # # 
2 | # # # # # # # # # # 
3 | # # # # # # # # # # 
4 | # # # # # # # # # # 
5 | # # # # # # # # # # 
6 | # # # # # # # # # # 
7 | # # # # # # # # # # 
8 | # # # # # # # # # # 
9 | # # # # # # # # # #